In [34]:
import pandas as pd
import json

## Load example parquets
The data folder can contain multiple parquete files, reading the folder path will open them together

In [41]:
# Load individual parquet

df_1 = pd.read_parquet("../data/example_data/example_data_1.parquet")
print(f"Number of samples in the the one example parquets: {len(df_1)}")


# Read folder of parquets
df = pd.read_parquet("../data/example_data/")

print(f"Number of samples in all parquets in the folder: {len(df)}")

Number of samples in the the one example parquets: 100
Number of samples in all parquets in the folder: 200


Specify columns for faster loading

In [43]:
df_single_column = pd.read_parquet("../data/example_data/example_data_1.parquet", columns=["smiles"])
display("Loaded parquete with only 'smiles' column")
display(df_single_column.head(2))


df_two_column = pd.read_parquet(
    "../data/example_data/example_data_1.parquet", columns=["smiles", "c_nmr_spectra"]
)
display("Loaded parquete with 'smiles' and 'c_nmr_spectra' columns")
df_two_column.head(2)

"Loaded parquete with only 'smiles' column"

,smiles
2228,CC(C)CCNc1ncc(F)cc1C(=O)O
1784,Fc1ccc(-c2cccs2)s1


"Loaded parquete with 'smiles' and 'c_nmr_spectra' columns"

,smiles,c_nmr_spectra
2228,CC(C)CCNc1ncc(F)cc1C(=O)O,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1784,Fc1ccc(-c2cccs2)s1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Load Meta data information

In [58]:
with open("../data/meta_data/meta_data_dict.json") as json_file:
    meta_data = json.load(json_file)
meta_data.keys()

dict_keys(['smiles', 'molecular_formula', 'h_nmr_spectra', 'c_nmr_spectra', 'h_nmr_peaks', 'c_nmr_peaks', 'hsqc_nmr_spectrum', 'hsqc_nmr_peaks', 'ir_spectra', 'msms_cfmid_positive_10ev', 'msms_cfmid_positive_20ev', 'msms_cfmid_positive_40ev', 'msms_cfmid_fragments_positive', 'msms_cfmid_negative_10ev', 'msms_cfmid_negative_20ev', 'msms_cfmid_negative_40ev', 'msms_cfmid_fragments_negative', 'msms_iceberg_positive', 'msms_iceberg_fragments_positive', 'msms_scarf_positive', 'msms_scarf_fragments_positive'])

In [59]:
h_nmr_peaks = df['h_nmr_spectra']
h_nmr_ppm = meta_data['h_nmr_spectra']['dimensions']